In [121]:
import pandas as pd
import datetime
import time
import math

In [122]:
pd.set_option('float_format', '{:f}'.format)
folder = 'WF/16_12_2018_19_01 f'
res_adapt = pd.read_csv(folder + '/adapt.csv')
res_adapt_eval = pd.read_csv(folder + '/adapt_eval.csv')
res_eval = pd.read_csv(folder + '/eval.csv')

In [123]:
res_adapt.drop(res_adapt[res_adapt.instance.str.contains('exact')].index, inplace=True)
res_eval.drop(res_eval[res_eval.instance.str.contains('exact')].index, inplace=True)
res_adapt_eval.drop(res_adapt_eval[res_adapt_eval.instance.str.contains('exact')].index, inplace=True)

In [124]:
res_corr = res_eval.merge(res_adapt, left_on='instance', right_on='instance', how='inner')

In [125]:
res_corr = res_corr.fillna(0)
res_corr['diff_p'] = (res_corr['pred_p'] - res_corr['real_p'])
res_corr['diff_p'] = res_corr['diff_p']*res_corr['diff_p']

In [126]:
res_corr

,Unnamed: 0_x,instance,type_x,k_x,real_p,pred_p,Unnamed: 0_y,type_y,k_y,old_p,old_r,old_f,new_p,new_r,new_f,diff_p
0,0,1 AMC Token Path Ontobuilder Max Delta,FeatureBased_SVM,-,0.463253,0.780319,0,IR,0,0.510861,0.756943,0.463253,0.496382,0.500000,0.498184,0.100531
1,0,1 AMC Token Path Ontobuilder Max Delta,FeatureBased_SVM,-,0.463253,0.780319,1,SVDpp,0,0.510861,0.756943,0.463253,0.663371,0.544792,0.569446,0.100531
2,0,1 AMC Token Path Ontobuilder Max Delta,FeatureBased_SVM,-,0.463253,0.780319,2,BPR,0,0.510861,0.756943,0.463253,0.510861,0.756943,0.463253,0.100531
3,0,1 AMC Token Path Ontobuilder Max Delta,FeatureBased_SVM,-,0.463253,0.780319,3,GRU,0,0.510861,0.756943,0.463253,0.512335,0.768540,0.473108,0.100531
4,0,1 AMC Token Path Ontobuilder Max Delta,FeatureBased_SVM,-,0.463253,0.780319,4,CNN,0,0.510861,0.756943,0.463253,0.496379,0.499669,0.498018,0.100531
5,0,1 AMC Token Path Ontobuilder Max Delta,FeatureBased_SVM,-,0.463253,0.780319,5,DNN,0,0.510861,0.756943,0.463253,0.496382,0.500000,0.498184,0.100531
6,0,1 AMC Token Path Ontobuilder Max Delta,FeatureBased_SVM,-,0.463253,0.780319,6,CRNN,0,0.510861,0.756943,0.463253,0.496360,0.497018,0.496689,0.100531
7,1,1 AMC Token Path Ontobuilder Max Delta,FeatureBased_SGD Regression,-,0.463253,2.618865,0,IR,0,0.510861,0.756943,0.463253,0.496382,0.500000,0.498184,4.646663
8,1,1 AMC Token Path Ontobuilder Max Delta,FeatureBased_SGD Regression,-,0.463253,2.618865,1,SVDpp,0,0.510861,0.756943,0.463253,0.663371,0.544792,0.569446,4.646663
9,1,1 AMC Token Path Ontobuilder Max Delta,FeatureBased_SGD Regression,-,0.463253,2.618865,2,BPR,0,0.510861,0.756943,0.463253,0.510861,0.756943,0.463253,4.646663


In [127]:
res_corr.groupby('type_x')[['pred_p', 'old_p', 'old_r', 'old_f']].corr()

pred_p     old_p  \
type_x                                                                  
CNN                                        pred_p  1.000000  0.113131   
                                           old_p   0.113131  1.000000   
                                           old_r   0.185680  0.507397   
                                           old_f   0.123284  0.875315   
CRNN                                       pred_p  1.000000  0.661182   
                                           old_p   0.661182  1.000000   
                                           old_r   0.132798  0.507397   
                                           old_f   0.840241  0.875315   
DNN                                        pred_p  1.000000  0.397757   
                                           old_p   0.397757  1.000000   
                                           old_r   0.178295  0.507397   
                                           old_f   0.303823  0.875315   
FeatureBased_Bayesian Ridge Regression     pred_p  1.000000  0.318939   
                                           old_p   0.318939  1.000000   
                                           old_r   0.684884  0.507397   
                                           old_f   0.329500  0.875315   
FeatureBased_Lasso Regression              pred_p  1.000000  0.318939   
                                           old_p   0.318939  1.000000   
                                           old_r   0.684884  0.507397   
                                           old_f   0.329500  0.875315   
FeatureBased_Linear Regression             pred_p  1.000000  0.318939   
                                           old_p   0.318939  1.000000   
                                           old_r   0.684884  0.507397   
                                           old_f   0.329500  0.875315   
FeatureBased_Passive Aggressive Regression pred_p  1.000000 -0.263854   
                                           old_p  -0.263854  1.000000   
                                           old_r   0.063832  0.507397   
                                           old_f  -0.256556  0.875315   
FeatureBased_SGD Regression                pred_p  1.000000 -0.288787   
                                           old_p  -0.288787  1.000000   
                                           old_r   0.025122  0.507397   
                                           old_f  -0.285681  0.875315   
FeatureBased_SVM                           pred_p  1.000000  0.318939   
                                           old_p   0.318939  1.000000   
                                           old_r   0.684884  0.507397   
                                           old_f   0.329500  0.875315   
FeatureBased_Theil-Sen Regression          pred_p  1.000000 -0.266372   
                                           old_p  -0.266372  1.000000   
                                           old_r   0.060080  0.507397   
                                           old_f  -0.259479  0.875315   
GRU                                        pred_p  1.000000  0.391153   
                                           old_p   0.391153  1.000000   
                                           old_r   0.019782  0.507397   
                                           old_f   0.354106  0.875315   

                                                     old_r     old_f  
type_x                                                                
CNN                                        pred_p 0.185680  0.123284  
                                           old_p  0.507397  0.875315  
                                           old_r  1.000000  0.543978  
                                           old_f  0.543978  1.000000  
CRNN                                       pred_p 0.132798  0.840241  
                                           old_p  0.507397  0.875315  
                                           old_r  1.000000  0.543978  
                                           old_f  0.543

In [128]:
res_adapt['diff_p'] = res_adapt['new_p'] - res_adapt['old_p']
res_adapt['diff_r'] = res_adapt['new_r'] - res_adapt['old_r']
res_adapt['diff_f'] = res_adapt['new_f'] - res_adapt['old_f']

In [129]:
res_adapt.groupby('type').mean()

,Unnamed: 0,k,old_p,old_r,old_f,new_p,new_r,new_f,diff_p,diff_r,diff_f
type,,,,,,,,,,,
BPR,9196.500000,0.000000,0.510357,0.572868,0.489646,0.120920,0.106809,0.041354,-0.389438,-0.466059,-0.448291
CNN,9198.500000,0.000000,0.510357,0.572868,0.489646,0.544594,0.554390,0.535658,0.034237,-0.018477,0.046012
CRNN,9200.500000,0.000000,0.510357,0.572868,0.489646,0.541660,0.546750,0.521264,0.031303,-0.026118,0.031618
DNN,9199.500000,0.000000,0.510357,0.572868,0.489646,0.544347,0.542798,0.535966,0.033989,-0.030070,0.046320
GRU,9197.500000,0.000000,0.510357,0.572868,0.489646,0.546142,0.572123,0.539621,0.035784,-0.000745,0.049975
IR,9194.500000,0.000000,0.510357,0.572868,0.489646,0.494517,0.500000,0.497238,-0.015840,-0.072868,0.007592
SVDpp,9195.500000,0.000000,0.510357,0.572868,0.489646,0.497057,0.502053,0.491530,-0.013300,-0.070815,0.001884


In [130]:
res_adapt['pair'], res_adapt['alg'] = res_adapt['instance'].str.split(' ', 1).str

In [132]:
res_adapt['new_f_max'] = res_adapt.groupby(['pair', 'type'])['new_f'].transform(max)
clean_a = res_adapt[res_adapt['new_f_max'] == res_adapt['new_f']]
clean_a.groupby('type').mean()
# res_adapt['diff_f_max'] = res_adapt.groupby(['pair'])['diff_f'].transform(max)
# clean_a = res_adapt[res_adapt['diff_f_max'] == res_adapt['diff_f']]
# clean_a.groupby('type').mean()
# clean_a.groupby('type').mean()
# grouped = clean_a[clean_a['type'] == 'CNN'].groupby(['type', 'pair']).mean().reset_index()
# grouped.sort_values('diff_f', ascending=False)

,Unnamed: 0,k,old_p,old_r,old_f,new_p,new_r,new_f,diff_p,diff_r,diff_f,new_f_max
type,,,,,,,,,,,,
BPR,8306.876712,0.000000,0.504827,0.594869,0.373800,0.499445,0.588302,0.367083,-0.005382,-0.006568,-0.006717,0.367083
CNN,3810.990050,0.000000,0.646211,0.740354,0.644477,0.729729,0.715264,0.706723,0.083518,-0.025090,0.062246,0.706723
CRNN,3964.323232,0.000000,0.651197,0.750034,0.646826,0.733820,0.713100,0.705204,0.082623,-0.036934,0.058378,0.705204
DNN,5368.113074,0.000000,0.593532,0.688327,0.579414,0.675956,0.641768,0.643702,0.082423,-0.046559,0.064288,0.643702
GRU,2773.130435,0.000000,0.700343,0.806495,0.714390,0.763683,0.799419,0.763293,0.063340,-0.007076,0.048903,0.763293
IR,9194.500000,0.000000,0.510357,0.572868,0.489646,0.494517,0.500000,0.497238,-0.015840,-0.072868,0.007592,0.497238
SVDpp,10312.840841,0.000000,0.495115,0.546978,0.480709,0.517126,0.518546,0.514840,0.022012,-0.028431,0.034131,0.514840


In [133]:
CRNN_adapt = res_adapt[res_adapt['type'] =='CRNN']

In [134]:
CRNN_adapt['new_f_max'] = CRNN_adapt.groupby(['pair'])['new_f'].transform(max)

C:\Users\shrag\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [135]:
CRNN_adapt_max = CRNN_adapt[CRNN_adapt['new_f_max'] == CRNN_adapt['new_f']]

In [137]:
CRNN_adapt_max.mean()

Unnamed: 0   3964.323232
k               0.000000
old_p           0.651197
old_r           0.750034
old_f           0.646826
new_p           0.733820
new_r           0.713100
new_f           0.705204
diff_p          0.082623
diff_r         -0.036934
diff_f          0.058378
pair                 inf
new_f_max       0.705204
dtype: float64

In [138]:
res_adapt_eval['pred_e_max'] = res_adapt_eval.groupby(['instance','type'])['real_e'].transform(max)
res_adapt_eval['k_max'] = res_adapt_eval.groupby(['instance','type'])['k'].transform(max)
clean_a_e = res_adapt_eval[(res_adapt_eval['pred_e_max'] == res_adapt_eval['real_e']) &
               (res_adapt_eval['k_max'] == res_adapt_eval['k'])]

In [139]:
res_adapt_eval

,Unnamed: 0,instance,type,k,old_p,old_r,old_f,new_p,new_r,new_f,real_e,pred_e,pred_e_max,k_max
0,0,1 AMC Token Path Ontobuilder Max Delta,GRU_GRU,0,0.510861,0.756943,0.463253,0.512335,0.768540,0.473108,0.463253,0.458160,0.463253,1
1,1,1 AMC Token Path Ontobuilder Max Delta,GRU_GRU,1,0.512335,0.768540,0.473108,0.512335,0.768540,0.473108,0.463253,0.680636,0.463253,1
2,2,1 AMC Token Path Ontobuilder Max Delta,CNN_CNN,0,0.510861,0.756943,0.463253,0.496379,0.499669,0.498018,0.463253,0.899688,0.463253,1
3,3,1 AMC Token Path Ontobuilder Max Delta,CNN_CNN,1,0.496379,0.499669,0.498018,0.496379,0.499669,0.498018,0.463253,0.967738,0.463253,1
4,4,1 AMC Token Path Ontobuilder Max Delta,DNN_DNN,0,0.510861,0.756943,0.463253,0.496382,0.500000,0.498184,0.463253,0.674669,0.463253,0
5,5,1 AMC Token Path Ontobuilder Max Delta,CRNN_CRNN,0,0.510861,0.756943,0.463253,0.496360,0.497018,0.496689,0.463253,0.300762,0.463253,1
6,6,1 AMC Token Path Ontobuilder Max Delta,CRNN_CRNN,1,0.496360,0.497018,0.496689,0.496360,0.497018,0.496689,0.463253,0.814879,0.463253,1
7,7,1 AMC Token Path Ontobuilder Max Delta,GRU_CNN,0,0.510861,0.756943,0.463253,0.512335,0.768540,0.473108,0.463253,0.899688,0.463253,0
8,8,1 AMC Token Path Ontobuilder Max Delta,GRU_DNN,0,0.510861,0.756943,0.463253,0.512335,0.768540,0.473108,0.463253,0.674669,0.463253,1
9,9,1 AMC Token Path Ontobuilder Max Delta,GRU_DNN,1,0.512335,0.768540,0.473108,0.512335,0.768540,0.473108,0.463253,0.704076,0.463253,1


In [140]:
clean_a_e.groupby('type').mean()[['new_p', 'new_r', 'new_f']]

,new_p,new_r,new_f
type,,,
BPR_FEATBayesian Ridge Regression,0.510357,0.572868,0.489646
BPR_FEATLasso Regression,0.510357,0.572868,0.489646
BPR_FEATLinear Regression,0.510357,0.572868,0.489646
BPR_FEATPassive Aggressive Regression,0.510357,0.572868,0.489646
BPR_FEATSGD Regression,0.510357,0.572868,0.489646
BPR_FEATSVM,0.510357,0.572868,0.489646
BPR_FEATTheil-Sen Regression,0.510357,0.572868,0.489646
BPR_GRU,0.510357,0.572868,0.489646
CNN_CNN,0.544619,0.554299,0.535674


In [141]:
res_adapt_eval['pair'], res_adapt_eval['alg'] = res_adapt_eval['instance'].str.split(' ', 1).str
CRNN_adapt_eval = res_adapt_eval[res_adapt_eval['type'] =='CRNN']

In [142]:
CRNN_adapt_eval['new_f_max'] = CRNN_adapt_eval.groupby(['pair'])['new_f'].transform(max)

In [143]:
CRNN_adapt_max = CRNN_adapt[CRNN_adapt['new_f_max'] == CRNN_adapt['new_f']]

In [146]:
CRNN_adapt_max.mean()

Unnamed: 0   3964.323232
k               0.000000
old_p           0.651197
old_r           0.750034
old_f           0.646826
new_p           0.733820
new_r           0.713100
new_f           0.705204
diff_p          0.082623
diff_r         -0.036934
diff_f          0.058378
pair                 inf
new_f_max       0.705204
dtype: float64

In [145]:
def summerize(data, file):
    idMax = data[data['type'] == data['type']]
    if 'pred_e' in data:
        idMax = data.groupby(['type'])['pred_e'].transform(max) == data['pred_e']
        pd.DataFrame(data[idMax].groupby(by='type').mean()).to_csv(file)
    else:
        pd.DataFrame(data.groupby(by='type').mean()).to_csv(file)

In [23]:
data = pd.read_csv('adapt.csv', low_memory=False)
summerize(data, 'a.csv')

FileNotFoundError: File b'adapt.csv' does not exist